/* Q1: Some of the facilities charge a fee to members, but some do not.
Write a SQL query to produce a list of the names of the facilities that do. */

            SELECT *
            FROM `Facilities`
            WHERE `membercost` !=0

/* Q2: How many facilities do not charge a fee to members? */

            SELECT COUNT( * )
            FROM Facilities
            WHERE membercost =0

/* Q3: Write an SQL query to show a list of facilities that charge a fee to members,
where the fee is less than 20% of the facility's monthly maintenance cost.
Return the facid, facility name, member cost, and monthly maintenance of the
facilities in question. */

            SELECT *
            FROM `Facilities`
            WHERE membercost < ( .2 * monthlymaintenance )

/* Q4: Write an SQL query to retrieve the details of facilities with ID 1 and 5.
Try writing the query without using the OR operator. */

            SELECT *
            FROM `Facilities`
            WHERE facid
            IN ( 1, 5 )

/* Q5: Produce a list of facilities, with each labelled as
'cheap' or 'expensive', depending on if their monthly maintenance cost is
more than $100. Return the name and monthly maintenance of the facilities
in question. */

            SELECT name, monthlymaintenance,
            CASE
            WHEN monthlymaintenance >100
            THEN 'expensive'
            ELSE 'cheap'
            END AS price
            FROM `Facilities`

/* Q6: You'd like to get the first and last name of the last member(s)
who signed up. Try not to use the LIMIT clause for your solution. */


            SELECT `firstname` , `surname` , MAX( `joindate` )
            FROM `Members`

/* Q7: Produce a list of all members who have used a tennis court.
Include in your output the name of the court, and the name of the member
formatted as a single column. Ensure no duplicate data, and order by
the member name. */

            SELECT DISTINCT concat(m.firstname,' ', m.surname) as member, f.name as facility
            FROM Members AS m
            INNER JOIN Bookings AS b
            USING ( memid )
            INNER JOIN Facilities AS f ON b.facid = f.facid
            WHERE f.name LIKE '%tennis court%'
            ORDER BY member;


/* Q8: Produce a list of bookings on the day of 2012-09-14 which
will cost the member (or guest) more than $30. Remember that guests have
different costs to members (the listed costs are per half-hour 'slot'), and
the guest user's ID is always 0. Include in your output the name of the
facility, the name of the member formatted as a single column, and the cost.
Order by descending cost, and do not use any subqueries. */


            SELECT CONCAT( m.firstname, ' ', m.surname ) AS member, f.name AS facility,
            CASE
            WHEN m.memid =0
            THEN b.slots * f.guestcost
            ELSE b.slots * f.membercost
            END AS cost
            FROM Members AS m
            INNER JOIN Bookings AS b ON m.memid = b.memid
            INNER JOIN Facilities AS f ON b.facid = f.facid
            WHERE b.starttime >= '2012-09-14'
            AND b.starttime < '2012-09-15'
            AND (
            (
            m.memid =0
            AND b.slots * f.guestcost >30
            )
            OR (
            m.memid !=0
            AND b.slots * f.membercost >30
            )
            )
            ORDER BY cost DESC

/* Q9: This time, produce the same result as in Q8, but using a subquery. */

            SELECT facility, member, cost
            FROM (

            SELECT m.surname AS member, f.name AS facility,
            CASE WHEN m.memid =0
            THEN b.slots * f.guestcost
            ELSE b.slots * f.membercost
            END AS cost
            FROM Members AS m
            INNER JOIN Bookings AS b
            USING ( memid )
            INNER JOIN Facilities AS f
            USING ( facid )
            WHERE b.starttime >= '2012-09-14'
            AND b.starttime < '2012-09-15'
            ) AS booking
            WHERE cost >30
            ORDER BY cost DESC

Part 2 : SQLite

In [2]:
import sqlite3
from sqlite3 import Error
import os.path

In [3]:
os.getcwd()

'/Users/regan/Documents/SQLFiles Tier 1'

In [4]:
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
    
    return conn


In [5]:
def select_all_tasks(conn):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    
    query1 = """
        SELECT *
        FROM Facilities"""
    cur.execute(query1)
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)
    return 




In [8]:
def select_all_tasks(conn):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    
    query1 = """
        SELECT name as FacilityName, totalrev as Revenue
FROM (
    SELECT f.name, SUM(
        CASE
        WHEN memid =0
        THEN b.slots * f.guestcost
        ELSE b.slots * f.membercost END ) AS totalrev
    FROM Bookings AS b
    INNER JOIN Facilities AS f ON b.facid = f.facid
    GROUP BY f.facid) AS revenue
WHERE totalrev < 1000
ORDER BY totalrev DESC"""
    cur.execute(query1)
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)
    return 



In [9]:
def main():
    database = "sqlite_db_pythonsqlite.db"
 #changed the above line to _db_##
    # create a database connection
    conn = create_connection(database)
    with conn: 
        print("2. Query all tasks")
        select_all_tasks(conn)
    return

In [10]:
if __name__ == '__main__':
    main()

2.6.0
2. Query all tasks
('Pool Table', 270)
('Snooker Table', 240)
('Table Tennis', 180)


/* Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order */

In [18]:
def select_all_tasks(conn):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    
    query1 = """
        SELECT Mem.surname, Mem.firstname, Mem.recommendedby, RecSurname, RecFirstName
        FROM Members AS Mem
        INNER JOIN (
            SELECT m1.surname AS RecSurname, m1.firstname AS RecFirstName, m1.recommendedby, m1.memid
            FROM Members AS m1
            INNER JOIN Members AS m2
            USING ( memid )
            ) AS rec 
        ON Mem.recommendedby = rec.memid
        ORDER BY Mem.surname, Mem.firstname"""
    cur.execute(query1)
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)
    return 
def main():
    database = "sqlite_db_pythonsqlite.db"
 #changed the above line to _db_##
    # create a database connection
    conn = create_connection(database)
    with conn: 
        print("2. Query all tasks")
        select_all_tasks(conn)
    return
if __name__ == '__main__':
    main()

2.6.0
2. Query all tasks
('Bader', 'Florence', '9', 'Stibbons', 'Ponder')
('Baker', 'Anne', '9', 'Stibbons', 'Ponder')
('Baker', 'Timothy', '13', 'Farrell', 'Jemima')
('Boothe', 'Tim', '3', 'Rownam', 'Tim')
('Butters', 'Gerald', '1', 'Smith', 'Darren')
('Coplin', 'Joan', '16', 'Baker', 'Timothy')
('Crumpet', 'Erica', '2', 'Smith', 'Tracy')
('Dare', 'Nancy', '4', 'Joplette', 'Janice')
('Genting', 'Matthew', '5', 'Butters', 'Gerald')
('Hunt', 'John', '30', 'Purview', 'Millicent')
('Jones', 'David', '4', 'Joplette', 'Janice')
('Jones', 'Douglas', '11', 'Jones', 'David')
('Joplette', 'Janice', '1', 'Smith', 'Darren')
('Mackenzie', 'Anna', '1', 'Smith', 'Darren')
('Owen', 'Charles', '1', 'Smith', 'Darren')
('Pinker', 'David', '13', 'Farrell', 'Jemima')
('Purview', 'Millicent', '2', 'Smith', 'Tracy')
('Rumney', 'Henrietta', '20', 'Genting', 'Matthew')
('Sarwin', 'Ramnaresh', '15', 'Bader', 'Florence')
('Smith', 'Jack', '1', 'Smith', 'Darren')
('Stibbons', 'Ponder', '6', 'Tracy', 'Burton')
('

this is the query I ran in PHP. "concat" threw an error for some reason

SELECT Mem.surname, Mem.firstname, Mem.recommendedby, RecommendedByName<br>
FROM Members AS Mem<br>
INNER JOIN (<br>
    SELECT CONCAT( m1.surname, ' ', m1.firstname ) AS RecommendedByName, m1.firstname, m1.recommendedby, m1.memid<br>
    FROM Members AS m1<br>
    INNER JOIN Members AS m2<br>
    USING ( memid )<br>
    ) AS rec <br>
    ON Mem.recommendedby = rec.memid <br>
ORDER BY surname, firstname <br>

/* Q12: Find the facilities with their usage by member, but not guests */

I interpretted this two ways:<br>
1) total usage by members, and (code in query) <br>
2) usage of each facility by each member (code in box below query) <br>

In [19]:
def select_all_tasks(conn):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    
    query1 = """
       SELECT COUNT( b.bookid ) AS Memberbookings, f.name
FROM Bookings AS b
INNER JOIN Facilities AS f
USING ( facid )
WHERE memid !=0
GROUP BY f.name"""
    cur.execute(query1)
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)
    return 
def main():
    database = "sqlite_db_pythonsqlite.db"
 #changed the above line to _db_##
    # create a database connection
    conn = create_connection(database)
    with conn: 
        print("2. Query all tasks")
        select_all_tasks(conn)
    return
if __name__ == '__main__':
    main()

2.6.0
2. Query all tasks
(344, 'Badminton Court')
(421, 'Massage Room 1')
(27, 'Massage Room 2')
(783, 'Pool Table')
(421, 'Snooker Table')
(195, 'Squash Court')
(385, 'Table Tennis')
(308, 'Tennis Court 1')
(276, 'Tennis Court 2')


Code for bookings by facility and individual member: <br>
SELECT COUNT( b.bookid ) AS totalbookings, b.memid, b.facid, f.name<br>
FROM Bookings AS b<br>
INNER JOIN Facilities AS f<br>
USING ( facid )<br>
WHERE memid !=0<br>
GROUP BY f.name, b.memid<br>


/* Q13: Find the facilities usage by month, but not guests */

In [22]:
def select_all_tasks(conn):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    
    query1 = """
       SELECT COUNT( b.bookid ) AS Memberbookings, f.name, strftime('%m', starttime) as Month 
FROM Bookings AS b
INNER JOIN Facilities AS f
USING ( facid )
WHERE memid !=0
GROUP BY Month"""
    cur.execute(query1)
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)
    return 
def main():
    database = "sqlite_db_pythonsqlite.db"
 #changed the above line to _db_##
    # create a database connection
    conn = create_connection(database)
    with conn: 
        print("2. Query all tasks")
        select_all_tasks(conn)
    return
if __name__ == '__main__':
    main()

2.6.0
2. Query all tasks
(480, 'Table Tennis', '07')
(1168, 'Tennis Court 1', '08')
(1512, 'Tennis Court 1', '09')
